In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
test_df = pd.read_csv('',index=False, encoding='utf-8-sig')

In [ ]:
# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
# test_df는 "재발방지대책 및 향후조치계획"와 "pred" 컬럼을 모두 가지고 있다고 가정합니다.
# 각 컬럼의 텍스트 리스트 추출
plans = test_df['재발방지대책 및 향후조치계획'].tolist()
preds = test_df['pred'].tolist()

# 두 컬럼의 임베딩 벡터 추출
embeddings_plans = model.encode(plans)
embeddings_preds = model.encode(preds)

# Cosine Similarity 계산 함수
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# 각 행에 대해 cosine similarity 계산
similarity_scores = [cosine_similarity(plan_emb, pred_emb) 
                     for plan_emb, pred_emb in zip(embeddings_plans, embeddings_preds)]

# 결과를 새로운 컬럼에 저장
test_df['cosine_similarity'] = similarity_scores

# 결과 출력
print(test_df[['재발방지대책 및 향후조치계획', 'pred', 'cosine_similarity']])
